<a href="https://colab.research.google.com/github/Bipin-Gouda/ML_Scratch/blob/main/Lab_6_CART_ALGO_FOR_LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pprint
from collections import Counter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_excel("/content/drive/MyDrive/Copy of Lab 4 ML Dataset - ID3.xlsx",index_col=None)
df.columns=df.iloc[1,:]
df=df.iloc[2:16,:]
print(df)

1      age  income student credit_rating Answer
2    young    high      no          fair     no
3    young    high      no     excellent     no
4   Midage    high      no          fair    yes
5   senior  medium      no          fair    yes
6   senior     low     yes          fair    yes
7   senior     low     yes     excellent     no
8   Midage     low     yes     excellent    yes
9    young  medium      no          fair     no
10   young     low     yes          fair    yes
11  senior  medium     yes          fair    yes
12   young  medium     yes     excellent    yes
13  Midage  medium      no     excellent    yes
14  Midage    high     yes          fair    yes
15  senior  medium      no     excellent     no


In [ ]:
X_train=df.iloc[:,:-1].values
Y_train=df.iloc[:,-1].values

In [ ]:
test = {
    'age': ['young', 'Midage'],
    'income': ['high', 'low'],
    'student': ['yes', 'no'],
    'credit_rating': ['excellent', 'excellent']
}
X_test = test
test = pd.DataFrame(X_test, columns=test.keys())
X_test

{'age': ['young', 'Midage'],
 'income': ['high', 'low'],
 'student': ['yes', 'no'],
 'credit_rating': ['excellent', 'excellent']}

In [ ]:
ytrain=[]
for i in Y_train:
  if i=='yes':
    ytrain.append(1)
  else:
    ytrain.append(0)
ytrain=np.array(ytrain)
print(ytrain)


[0 0 1 1 1 0 1 0 1 1 1 1 1 0]


In [ ]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y, depth=0):
        if depth == 0:
            self.classes = np.unique(y)
        if (self.max_depth is not None and depth >= self.max_depth) or np.all(y == y[0]):
#             print(f"Reached leaf node with class {y[0]}")
            return self.classes[np.argmax(np.bincount(y))]

        best_split = self.find_best_split(X, y)
        if best_split is None:
#             print("No further split is possible.")
            return self.classes[np.argmax(np.bincount(y))]

        feature_idx, threshold = best_split
#         print(f"Splitting on feature {feature_idx} with value {threshold}")
        sub_tree = {f"X{feature_idx} = {threshold}": {}, f"noX{feature_idx} = {threshold}": {}}
        X_left, y_left, X_right, y_right = self.split_data(X, y, feature_idx, threshold)

        sub_tree[f"X{feature_idx} = {threshold}"] = self.fit(X_left, y_left, depth + 1)
        sub_tree[f"noX{feature_idx} = {threshold}"] = self.fit(X_right, y_right, depth + 1)

        return sub_tree

    def find_best_split(self, X, y):
        num_samples, num_features = X.shape
        if num_samples <= 1:
            return None

        gini_parent = self.calculate_gini(y)
        best_ginich = 0
        best_split = None

        for feature_idx in range(num_features):
            unique_values = np.unique(X[:, feature_idx])
            for threshold in unique_values:
                X_left, y_left, X_right, y_right = self.split_data(X, y, feature_idx, threshold)
                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                gini_left = self.calculate_gini(y_left)
                gini_right = self.calculate_gini(y_right)
                gini_change = gini_parent - (len(y_left) / num_samples * gini_left + len(y_right) / num_samples * gini_right)

                if gini_change > best_ginich:
                    best_ginich = gini_change
                    best_split = (feature_idx, threshold)

        return best_split

    def split_data(self, X, y, feature_idx, threshold):
        X_left = X[X[:, feature_idx] == threshold]
        y_left = y[X[:, feature_idx] == threshold]
        X_right = X[X[:, feature_idx] != threshold]
        y_right = y[X[:, feature_idx] != threshold]
        return X_left, y_left, X_right, y_right

    def calculate_gini(self, y):
        if len(y) == 0:
            return 0
        class_counts = Counter(y)
        num_samples = len(y)
        entropy = 1-sum((count / num_samples)**2 for count in class_counts.values())
        return entropy


    def predict(self, X, tree):
        if isinstance(tree, np.ndarray):
            return tree
        if isinstance(tree, dict):

                condition=list(tree.keys())[0]
                feature_idx, threshold = map(str.strip, condition.split("X")[1].split("="))
                feature_idx = int(feature_idx)

                if X[feature_idx] == threshold:

                    return self.predict(X, tree[f"X{feature_idx} = {threshold}"])
                else:

                    return self.predict(X, tree[f"noX{feature_idx} = {threshold}"])
        else:
            return tree

    def fit_predict(self, X_train, y_train, X_test):
        self.tree = self.fit(X_train, y_train)
        pprint.pprint(self.tree)
        train_predictions = [self.predict(x, self.tree) for x in X_train]
        test_predictions = [self.predict(x, self.tree) for x in X_test]
        return train_predictions, test_predictions

In [ ]:
# Building Tree

In [ ]:
decision_tree = DecisionTree(max_depth=8)
train_predictions, test_predictions = decision_tree.fit_predict(X_train, ytrain, X_test)

{'X0 = Midage': 1,
 'noX0 = Midage': {'X2 = no': {'X0 = senior': {'X3 = excellent': 0,
                                               'noX3 = excellent': 1},
                               'noX0 = senior': 0},
                   'noX2 = no': {'X3 = excellent': {'X0 = senior': 0,
                                                    'noX0 = senior': 1},
                                 'noX3 = excellent': 1}}}


IndexError: ignored

In [ ]:
decision_tree

In [ ]:
# Prdeiction
print(test_predictions)